In [45]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras import backend as K
from keras.applications.vgg16 import VGG16

# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data_expanded/train'
validation_data_dir = 'data_expanded/validation'
epochs = 10
batch_size = 50

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


model_vgg = VGG16(input_shape=input_shape, include_top=False)

model = Sequential()

model.add(model_vgg)
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(64))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

model.add(top_model)
print(len(top_model.layers))
print(len(model.layers))

model.layers[0].trainable = False 

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

top_model.save_weights("./top_model_pre_weights.h5")

Found 397 images belonging to 2 classes.
Found 377 images belonging to 2 classes.
6
2
Epoch 1/10
40/40 [==============================] - 631s 16s/step - loss: 1.2020 - acc: 0.7626 - val_loss: 0.1876 - val_acc: 0.9310
Epoch 2/10
40/40 [==============================] - 621s 16s/step - loss: 0.1783 - acc: 0.9341 - val_loss: 0.2572 - val_acc: 0.8992
Epoch 3/10
40/40 [==============================] - 624s 16s/step - loss: 0.1097 - acc: 0.9608 - val_loss: 0.1964 - val_acc: 0.9257
Epoch 4/10
40/40 [==============================] - 655s 16s/step - loss: 0.0737 - acc: 0.9738 - val_loss: 0.2308 - val_acc: 0.9125
Epoch 5/10
40/40 [==============================] - 634s 16s/step - loss: 0.0499 - acc: 0.9865 - val_loss: 0.1928 - val_acc: 0.9204
Epoch 6/10
40/40 [==============================] - 637s 16s/step - loss: 0.0467 - acc: 0.9849 - val_loss: 0.2326 - val_acc: 0.9204
Epoch 7/10
40/40 [==============================] - 626s 16s/step - loss: 0.0292 - acc: 0.9925 - val_loss: 0.2687 - val_ac

[[1.7958418e-04 1.5440178e-03 6.2330737e-04 ... 2.8651350e-05
  1.6895305e-04 4.7039956e-02]
 [2.4859858e-04 4.1794558e-03 9.8428049e-04 ... 6.9466128e-05
  2.4243415e-04 2.5978614e-02]
 [1.3468292e-04 1.6860118e-03 7.9931872e-04 ... 3.1812764e-05
  1.9653772e-04 5.2003644e-02]
 ...
 [2.8982840e-04 5.0914055e-03 9.9668477e-04 ... 7.4937838e-05
  2.5978414e-04 2.6783589e-02]
 [2.8772210e-04 2.0660767e-03 7.2282058e-04 ... 4.6983769e-05
  2.3661484e-04 3.6839090e-02]
 [1.8017448e-04 2.2002356e-03 5.8848609e-04 ... 3.1118430e-05
  1.9233243e-04 4.8730135e-02]]
